In [25]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from pydantic import Field, BaseModel
import operator

In [26]:
load_dotenv()

True

In [27]:
model= ChatOpenAI(model="gpt-4o-mini")

In [28]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description="Detailed feedback about the essay")
    score: int = Field(description="Score Out of 10", le=10, ge=0)

In [29]:
structured_model= model.with_structured_output(EvaluationSchema)

In [30]:
essay= """
The rapid evolution of Artificial Intelligence has brought us to the precipice of a new era: Agentic AI. Moving beyond mere data analysis or content generation, agentic AI systems are designed to act autonomously, make decisions, and execute complex tasks with minimal human intervention. This shift from passive assistance to proactive action heralds a profound transformation across industries and society at large, leading to the undeniable conclusion that agentic AI will, indeed, fundamentally change the world.

At its core, agentic AI's transformative power lies in its ability to automate entire workflows, not just individual tasks. Unlike traditional AI, which relies on predefined rules, or even generative AI, which primarily creates content, agentic AI leverages large language models (LLMs) as a "brain" to set goals, plan, and then utilize various tools and other AI agents to achieve those objectives. This autonomy allows for unprecedented efficiency and scalability. In software development, agentic AI can automate code generation, debugging, and testing, accelerating the development cycle. In legal services, it can parse vast databases, draft contracts, and ensure compliance. Customer service is already witnessing a revolution, with agentic AI predicted to autonomously resolve a significant majority of common issues, leading to substantial cost reductions and improved customer satisfaction.

The impact extends far beyond commercial applications. In healthcare, agentic AI promises to assist in diagnosis, treatment planning, and drug discovery by analyzing immense datasets. Supply chain management can be optimized through predictive analytics and automated logistics. Even in complex domains like urban planning and environmental management, agentic AI could simulate scenarios, identify optimal solutions, and execute interventions. The potential for increased productivity, innovation, and problem-solving at a scale previously unimaginable is immense. Entire departments, and even some functions, could theoretically become "zero-FTE" as AI agents take on a vast array of responsibilities, freeing human employees to focus on more creative, strategic, and emotionally intelligent tasks.

However, such a seismic shift is not without its challenges and risks. The enhanced autonomy of agentic AI systems raises critical questions about accountability and responsibility. When an autonomous system makes a flawed decision, pinpointing who is to blame – the developer, the deployer, or the user – becomes a complex ethical and legal labyrinth. The "black box" nature of many advanced AI systems, where their internal reasoning is opaque, further complicates auditing and understanding their decisions, potentially eroding trust. Moreover, the pervasive use of agentic AI could exacerbate existing societal inequalities if access to these advanced technologies is not equitable, or if biased training data leads to discriminatory outcomes in areas like financial services or employment. Job displacement, particularly in sectors reliant on routine tasks, is another significant concern, necessitating robust reskilling and upskilling initiatives.

Ultimately, the future shaped by agentic AI will depend on how we navigate these profound opportunities and challenges. To harness its full potential for positive change, a multi-faceted approach is required. This includes developing clear ethical guidelines and regulatory frameworks, investing heavily in education and workforce retraining programs, ensuring data privacy and cybersecurity, and fostering a collaborative environment where humans and AI work synergistically. The notion of "superagency," where AI amplifies human capabilities rather than replacing them entirely, offers a compelling vision for this future.

In conclusion, agentic AI is not merely an incremental technological advancement; it is a fundamental redefinition of how work is done, decisions are made, and problems are solved. Its ability to autonomously reason, plan, and act will undoubtedly reshape industries, economies, and our daily lives in ways that are only just beginning to be understood. While the path forward is fraught with ethical complexities and societal adjustments, the transformative potential of agentic AI to drive unprecedented efficiency, innovation, and progress is undeniable, making it a true world-changer.

"""

In [31]:
answer=structured_model.invoke(essay)

In [32]:
answer.score

8

In [33]:
class EssayState(TypedDict):
    essay: str
    language_fb: str
    analysis_fb: str
    clarity_fb: str
    overall_fb: str
    individual_score: Annotated[list[int], operator.add]
    avg_score: float

In [34]:
def EvaluateLanguage(state:EssayState):

    prompt= f"Evaluate the langauge quality of the following essay and provide the feedback and assign the socre out of 10.\n {state['essay']}"

    output= structured_model.invoke(prompt)

    return {'language_fb': output.feedback, 'individual_score':[output.score]}

In [35]:
def EvaluateAnalysis(state:EssayState):

    prompt= f"Evaluate the Depth of Analysis having in the following essay and provide the feedback and assign the socre out of 10.\n {state['essay']}"

    output= structured_model.invoke(prompt)

    return {'analysis_fb': output.feedback, 'individual_score':[output.score]}

In [36]:
def EvaluateThought(state:EssayState):

    prompt= f"Evaluate the Clarity of Thought having in the following essay and provide the feedback and assign the socre out of 10.\n {state['essay']}"

    output= structured_model.invoke(prompt)

    return {'clarity_fb': output.feedback, 'individual_score':[output.score]}

In [37]:
def FinalEvaluation(state:EssayState):
    prompt= f"Provide the summary of following feedback in simple manner:\n language feedback {state['language_fb']}, \n Clarity of Thoughts {state['clarity_fb']},\n Depth of Analysis {state['clarity_fb']}"

    output= model.invoke(prompt).content

    avg_score= sum(state['individual_score'])/len(state['individual_score'])

    return {'overall_fb':output, 'avg_score': avg_score}

In [38]:
graph= StateGraph(EssayState)

graph.add_node('EvaluateLanguage',EvaluateLanguage)
graph.add_node('EvaluateAnalysis', EvaluateAnalysis)
graph.add_node('EvaluateThought', EvaluateThought)
graph.add_node('FinalEvaluation', FinalEvaluation)

# Add Edges
graph.add_edge(START, 'EvaluateLanguage')
graph.add_edge(START, 'EvaluateAnalysis')
graph.add_edge(START, 'EvaluateThought')

graph.add_edge('EvaluateLanguage','FinalEvaluation')
graph.add_edge('EvaluateAnalysis','FinalEvaluation')
graph.add_edge('EvaluateThought','FinalEvaluation')

graph.add_edge('FinalEvaluation', END)

workflow= graph.compile()

In [40]:
print(workflow.get_graph().draw_ascii())

                                  +-----------+                                    
                                  | __start__ |**                                  
                             *****+-----------+  *****                             
                       ******            *            *****                        
                  *****                  *                 *****                   
               ***                       *                      ***                
+------------------+           +------------------+           +-----------------+  
| EvaluateAnalysis |           | EvaluateLanguage |           | EvaluateThought |  
+------------------+***        +------------------+        ***+-----------------+  
                       ******            *            *****                        
                             *****       *       *****                             
                                  ***    *    ***                           

In [41]:
workflow.invoke({'essay':essay})

{'essay': '\nThe rapid evolution of Artificial Intelligence has brought us to the precipice of a new era: Agentic AI. Moving beyond mere data analysis or content generation, agentic AI systems are designed to act autonomously, make decisions, and execute complex tasks with minimal human intervention. This shift from passive assistance to proactive action heralds a profound transformation across industries and society at large, leading to the undeniable conclusion that agentic AI will, indeed, fundamentally change the world.\n\nAt its core, agentic AI\'s transformative power lies in its ability to automate entire workflows, not just individual tasks. Unlike traditional AI, which relies on predefined rules, or even generative AI, which primarily creates content, agentic AI leverages large language models (LLMs) as a "brain" to set goals, plan, and then utilize various tools and other AI agents to achieve those objectives. This autonomy allows for unprecedented efficiency and scalability.

In [42]:
essay2= """
The w of bi of Alligence has brought us to the precipice of a new era: Agentic AI. Moving beyond mere data analysis or content generation, agentic AI systems are designed to act autonomously, make decisions, and execute complex tasks with minimal human intervention. This shift from passive assistance to proactive action heralds a profound transformation across industries and society at large, leading to the undeniable conclusion that agentic AI will, indeed, fundamentally change the world.

At its core, agentic AI's transformative power lies in its abi tasks. Unlike traditional AI, which relies on predef
The impact extends far beyond commercial apptonomy of
Ultimately, the future shaped by agentic AI will depend on positive change, a multi-teum approach is required.

In conclusion, agentic AI is not mereact will undoubtedly reshape industries, economies, and our daily lives in ways that are only just beginning to be understood. While the path forward is fraught with ethical complexities and societal adjustments, the transformative potential of agentic AI to drive unprecedented efficiency, innovation, and progress is undeniable, making it a true world-changer.

"""

In [43]:
workflow.invoke({'essay':essay2})

{'essay': "\nThe w of bi of Alligence has brought us to the precipice of a new era: Agentic AI. Moving beyond mere data analysis or content generation, agentic AI systems are designed to act autonomously, make decisions, and execute complex tasks with minimal human intervention. This shift from passive assistance to proactive action heralds a profound transformation across industries and society at large, leading to the undeniable conclusion that agentic AI will, indeed, fundamentally change the world.\n\nAt its core, agentic AI's transformative power lies in its abi tasks. Unlike traditional AI, which relies on predef\nThe impact extends far beyond commercial apptonomy of\nUltimately, the future shaped by agentic AI will depend on positive change, a multi-teum approach is required.\n\nIn conclusion, agentic AI is not mereact will undoubtedly reshape industries, economies, and our daily lives in ways that are only just beginning to be understood. While the path forward is fraught with 